In [2]:
!pip install -qU pinecone-client==3.2.1 pinecone-notebooks tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.3 requires pinecone-client<6.0.0,>=5.0.0, but you have pinecone-client 3.2.1 which is incompatible.


In [10]:
import os 
import time
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec


In [5]:
load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')

In [7]:
from src.helper import download_hugging_face_embeddings


In [8]:
embeddings = download_hugging_face_embeddings()


/opt/anaconda3/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec("aws","us-east-1")
index_name="medical-chatbot"

In [14]:
index = pc.Index(index_name)

In [15]:
# Check existing indexes
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    # Create index if it does not exist
    pc.create_index(index_name, dimension=384, metric='cosine', spec=spec)
    
    # Wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    
    print(f"Index {index_name} has been successfully created.")
else:
    print(f"Index {index_name} already exists.")


Index medical-chatbot already exists.


In [16]:
index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19008}},
 'total_vector_count': 19008}

In [1]:
from langchain.chains.combine_documents import create_stuff_documents_chain
